In [1]:
import pandas

import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')
import os

Mounted at /content/drive


In [2]:
import pandas as pd

# Specify the file path
file_path = '/content/drive/MyDrive/Austral/Labo 3/tb_sellout_01.txt'

# Read the lines of the text file
with open(file_path, 'r') as file:
    lines = file.readlines()

# Extract column names from the first 6 lines (assuming 6 lines for column names)
column_names = [line.strip() for line in lines[:6]]

# Create a dictionary from the data
data = {}
for i, col_name in enumerate(column_names):
    data[col_name] = [line.strip() for line in lines[6 + i::6]]

# Create the DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

        periodo customer_id product_id cust_request_qty cust_request_tn  \
0        201701       10234      20524                2           0.053   
1        201701       10032      20524                1         0.13628   
2        201701       10217      20524                1         0.03028   
3        201701       10125      20524                1         0.02271   
4        201701       10012      20524               11         1.54452   
...         ...         ...        ...              ...             ...   
2945813  201912       10105      20853                1          0.0223   
2945814  201912       10092      20853                1         0.00669   
2945815  201912       10006      20853                7         0.02898   
2945816  201912       10018      20853                4         0.01561   
2945817  201912       10020      20853                2         0.01561   

              tn  
0          0.053  
1        0.13628  
2        0.03028  
3        0.02271  
4   

In [3]:
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m')


In [4]:
df

,periodo,customer_id,product_id,cust_request_qty,cust_request_tn,tn
0,2017-01-01,10234,20524,2,0.053,0.053
1,2017-01-01,10032,20524,1,0.13628,0.13628
2,2017-01-01,10217,20524,1,0.03028,0.03028
3,2017-01-01,10125,20524,1,0.02271,0.02271
4,2017-01-01,10012,20524,11,1.54452,1.54452
...,...,...,...,...,...,...
2945813,2019-12-01,10105,20853,1,0.0223,0.0223
2945814,2019-12-01,10092,20853,1,0.00669,0.00669
2945815,2019-12-01,10006,20853,7,0.02898,0.02898
2945816,2019-12-01,10018,20853,4,0.01561,0.01561


In [5]:
df['tn'] = pd.to_numeric(df['tn'])
df['cust_request_tn'] = pd.to_numeric(df['cust_request_tn'])

In [6]:
df.dtypes

periodo             datetime64[ns]
customer_id                 object
product_id                  object
cust_request_qty            object
cust_request_tn            float64
tn                         float64
dtype: object

**Tengo que predecir las ventas del producto estrella 20.001 para abril de 2019**

In [7]:
df2 = df[df['product_id'] == '20001']
df2 = df2[["periodo", "tn"]]
df2 = df2.groupby('periodo').sum().reset_index() # Agrupa los datos por la columna "fecha" y suma las toneladas

In [8]:
fecha_referencia = pd.to_datetime('2019-02-01') # Crea una fecha de referencia para febrero de 2019
df2 = df2[df2['periodo'] <= fecha_referencia]

In [9]:
# Establece la columna "periodo" como el índice
df2 = df2.set_index('periodo')

In [12]:
!pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.0 MB/s eta 0:00:00


In [13]:
from pmdarima.arima import auto_arima

# Ajusta el modelo ARIMA automáticamente
modelo_arima = auto_arima(df2, seasonal=True, trace=True, error_action='ignore', suppress_warnings=True)
modelo_arima.fit(df2)

# Realiza la predicción para un período futuro (por ejemplo, los próximos 2 pasos)
prediccion = modelo_arima.predict(n_periods=2)
print (prediccion)


Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=382.639, Time=0.79 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=377.804, Time=0.07 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=377.670, Time=0.19 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=378.211, Time=0.46 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=452.463, Time=0.06 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=379.528, Time=0.52 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=379.833, Time=0.17 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=381.101, Time=0.25 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=388.826, Time=0.08 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0] intercept
Total fit time: 2.655 seconds
2019-03-01    1331.579065
2019-04-01    1350.644094
Freq: MS, dtype: float64


In [14]:
prediccion = pd.DataFrame(prediccion)
prediccion.columns = ["tn_pred"]


In [15]:
df_test = df[df['product_id'] == '20001']
test = df_test[["periodo", "tn"]]
test = test.groupby('periodo').sum().reset_index()

# Filtra las filas correspondientes a marzo y abril de 2019
filtro = (test['periodo'] >= '2019-03-01') & (test['periodo'] <= '2019-04-30')
test = test[filtro]
test = test.set_index('periodo')

In [16]:
test

,tn
periodo,
2019-03-01,1470.65653
2019-04-01,1647.63848


In [17]:
# Calcular el error cuadrático medio (MSE) en el conjunto de prueba transformado
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(test, prediccion)
print("Error Cuadrático Medio (MsE):", mse)

# Calcular el error medio absoluto (MAE)
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(test, prediccion)
print("Error Absoluto Medio (MAE):", mae)

Error Cuadrático Medio (MsE): 53774.1032420452
Error Absoluto Medio (MAE): 218.035925403428


In [18]:
import numpy as np
#merge según index
dif = test.merge(prediccion, left_index=True, right_index=True, how='inner')
dif["error"] = dif['tn'] - dif['tn_pred']
dif

,tn,tn_pred,error
periodo,,,
2019-03-01,1470.65653,1331.579065,139.077465
2019-04-01,1647.63848,1350.644094,296.994386


In [19]:
sum_real = dif["tn"].sum()
sum_dif = dif["error"].sum()
error= sum_dif/sum_real
error

0.13984303903525022

REHACER PARA CADA PRODUCTO. VER FOTO

In [20]:
# Calcular el porcentaje de valores nulos en cada columna
porcentaje_nulos = (df.isnull().mean() * 100).round(2)
# Ordenar de mayor a menor porcentaje de nulos
porcentaje_nulos_sorted = porcentaje_nulos.sort_values(ascending=False)
print(porcentaje_nulos_sorted)

periodo             0.0
customer_id         0.0
product_id          0.0
cust_request_qty    0.0
cust_request_tn     0.0
tn                  0.0
dtype: float64


In [22]:
df2 = df[["periodo", "product_id", "tn"]]
df2 = df2.groupby(["periodo", "product_id"]).sum().reset_index()

fecha_referencia = pd.to_datetime('2019-02-01') # Crea una fecha de referencia para febrero de 2019
df2 = df2[df2['periodo'] <= fecha_referencia]

df2 = df2.set_index('periodo')

In [ ]:
# Obtén la lista única de categorías en la columna "product_id"
categorias = df2["product_id"].unique()

# Inicializa un diccionario para almacenar las predicciones
predicciones_por_categoria = {}

# Itera a través de cada categoría
for categoria in categorias:
    # Filtra el DataFrame para la categoría actual
    df_categoria = df2[df2["product_id"] == categoria]

    modelo_arima = auto_arima(df_categoria["tn"], seasonal=True, trace=True, error_action='ignore', suppress_warnings=True)
    modelo_arima.fit(df_categoria["tn"])

    # Realiza la predicción para un período futuro (por ejemplo, los próximos 2 pasos)
    prediccion = modelo_arima.predict(n_periods=2)

    # Almacena las predicciones en el diccionario
    predicciones_por_categoria[categoria] = prediccion

# Ahora, el diccionario predicciones_por_categoria contiene las predicciones para cada categoría
print(predicciones_por_categoria)


Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=382.639, Time=0.20 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=377.804, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=377.670, Time=0.06 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=378.211, Time=0.09 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=452.463, Time=0.02 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=379.528, Time=0.11 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=379.833, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=381.101, Time=0.09 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=388.826, Time=0.04 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0] intercept
Total fit time: 0.700 seconds
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.25 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=356.235, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=350.820, Time=0.06 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.13 sec


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=207.026, Time=0.05 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=207.093, Time=0.07 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=266.819, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=208.981, Time=0.10 sec

Best model:  ARIMA(0,0,0)(0,0,0)[0] intercept
Total fit time: 0.389 seconds
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.23 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=243.551, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=245.492, Time=0.06 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=245.500, Time=0.05 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=286.089, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=247.439, Time=0.07 sec

Best model:  ARIMA(0,0,0)(0,0,0)[0] intercept
Total fit time: 0.461 seconds
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.27 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=235.024, Ti

In [ ]:
import pandas as pd
from pmdarima.arima import auto_arima

# Supongamos que df2 es tu DataFrame

# Obtén la lista única de categorías en la columna "product_id"
categorias = df2["product_id"].unique()

# Inicializa un diccionario para almacenar las predicciones
predicciones_por_categoria = {}

# Itera a través de cada categoría
for categoria in categorias:
    # Filtra el DataFrame para la categoría actual
    df_categoria = df2[df2["product_id"] == categoria]

    # Asegúrate de que la columna de series temporales no contiene valores nulos
    if df_categoria["tu_columna_de_serie_temporal"].isnull().any():
        continue  # Salta la categoría si contiene valores nulos

    # Asegúrate de que el índice sea una serie temporal válida
    df_categoria["tn"] = pd.to_datetime(df_categoria["fecha"])  # Asumiendo que "fecha" es la columna de fechas
    df_categoria.set_index("tn", inplace=True)

    # Ajusta el modelo ARIMA automáticamente para la categoría actual
    y = df_categoria["tu_columna_de_serie_temporal"]  # Reemplaza "tu_columna_de_serie_temporal" con el nombre correcto de la columna de series temporales
    modelo_arima = auto_arima(y, seasonal=True, trace=True, error_action='ignore', suppress_warnings=True)
    modelo_arima.fit(y)

    # Realiza la predicción para un período futuro (por ejemplo, los próximos 2 pasos)
    prediccion = modelo_arima.predict(n_periods=2)

    # Almacena las predicciones en el diccionario
    predicciones_por_categoria[categoria] = prediccion

# Ahora, el diccionario predicciones_por_categoria contiene las predicciones para cada categoría sin valores nulos
print(predicciones_por_categoria)


In [ ]:
columnas_con_nan = df.isna()